**PORCENTAJE DE AGEBs URBANAS CON MÁS DEL 30% DE VIVIENDA DESTINADA A ALOJAMIENTOS DE RENTA A CORTO PLAZO**

In [4]:
#LIBRERIAS (instalación y carga)
libraries <- c("tidyverse",
               "sf", 
               "readr")
installed_packages <- rownames(installed.packages())
libraries_to_install <- setdiff(libraries, installed_packages)

if (length(libraries_to_install) > 0) {
  install.packages(libraries_to_install)}

lapply(libraries, library, character.only = TRUE)

Linking to GEOS 3.13.0, GDAL 3.10.1, PROJ 9.5.1; sf_use_s2() is TRUE



[[1]]
 [1] "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"     "readr"    
 [7] "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"     "graphics" 
[13] "grDevices" "utils"     "datasets"  "methods"   "base"     

[[2]]
 [1] "sf"        "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"    
 [7] "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     

[[3]]
 [1] "sf"        "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"    
 [7] "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"

In [ ]:
# #Áreas geoestadísticas básicas (AGEB) de la Ciudad de México del Marco Geoestadístico Nacional 2020
# url <- "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/09_ciudaddemexico.zip"

# temp_zip <- tempfile(fileext = ".zip")                                                  #Archivo temporal para descargar el zip    
# temp_dir <- tempdir()                                                                   #Directorio temporal para descomprimir el zip

# options(timeout = 8000)                                                                 #Tiempo de espera para la descarga 

# download.file(url, temp_zip, mode = "wb")                                               #Descarga el archivo zip en el directorio temporal

# related_files <- paste0("conjunto_de_datos/09a", c(".shp", ".shx", ".dbf", ".prj"))     #Busqueda de los archivos shp dentro del zip

# unzip(temp_zip, files = related_files, exdir = temp_dir)                                #Descomprime el zip en el directorio temporal

# shp_file_path <- file.path(temp_dir, related_files[1])                                  #Ruta del archivo shp

# ageb_2020 <- st_read(shp_file_path, quiet = TRUE)|>                                     #Carga el shapefile de las AGEBS
#   st_transform(crs = 32614)                                                             #Transformación el sistema de coordenadas a WGS84 UTM 14N


In [2]:
## PRINCIPALES RESULTADOS POR MANZANA CENSO DE POBLACIÓN Y VIVIENDA 2020 

url <- "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_09_2020_xlsx.zip"

temp_zip <- tempfile(fileext = ".zip")        # Archivo temporal para descargar el zip
temp_dir <- tempdir()                         # Directorio temporal para descomprimir el zip

options(timeout = 800)                        # Tiempo de espera para la descarga

download.file(url, temp_zip, mode = "wb")     # Descarga el archivo zip en el directorio temporal

unzip(temp_zip, exdir = temp_dir)             # Descomprime el zip en el directorio temporal

# Localiza el archivo Excel dentro del zip descomprimido
RESAGEBURB_09XLSX20_BASE <- list.files(
  temp_dir, 
  pattern = "RESAGEBURB_09XLSX20.xlsx", 
  full.names = TRUE, 
  recursive = TRUE
)

# Cargar el archivo Excel, limpiar y procesar
library(readxl)
library(dplyr)

RESAGEBURB_09XLSX20 <- read_excel(RESAGEBURB_09XLSX20_BASE) |>
  mutate(
    CVEGEO = paste0(ENTIDAD, MUN, LOC, AGEB)  # Crear clave geográfica
  ) |>
  mutate_all(~ gsub("\\*", "", .)) |>         # Eliminar asteriscos de todo el data frame
  mutate(across(9:230, ~ as.numeric(.))) |>   # Convertir variables 9 a 230 en numéricas
  filter(NOM_MUN == "Total de la entidad Ciudad de México")|>
  select(NOM_MUN,VIVTOT, TVIVPARHAB, TOTHOG, POBTOT)

RESAGEBURB_09XLSX20

Warning message:
"There were 217 warnings in `mutate()`.
The first warning was:
ℹ In argument: `across(9:230, ~as.numeric(.))`.
Caused by warning:
! NAs introducidos por coerción
ℹ Run `dplyr::last_dplyr_warnings()` to see the 216 remaining warnings."


NOM_MUN,VIVTOT,TVIVPARHAB,TOTHOG,POBTOT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Total de la entidad Ciudad de México,3036239,2756319,2756319,9209944


In [4]:
(17430/3036239)*100

[1] 0.5740655

In [ ]:
# pob<-ageb_2020|>
#   left_join(RESAGEBURB_09XLSX20, by = "CVEGEO")

In [ ]:
#DATOS AIRBNB (corte de la información a diciembre 2024)
url <- "https://data.insideairbnb.com/mexico/df/mexico-city/2024-12-27/data/listings.csv.gz"

airbnb_base<-read_csv(url, col_types = cols(.default = col_character()))|>            #Carga de la base                                                          
  mutate(latitude= as.numeric(latitude),                                                                                                    
         longitude= as.numeric(longitude))|>                                          #Convertir a numerico los campos de latitud y longitud
  filter(room_type == "Entire home/apt")

cuenta_airbnb <- airbnb_base|>
group_by(room_type)|>
summarise(alojamientos = n())

cuenta_airbnb$alojamientos


#                                              #Seleccionar el tipo de alojamiento (aquellos que son propiedades completas)
#   st_as_sf(coords = c("longitude", "latitude"), crs = 4326)|>                         #Transformar a objeto espacial                  
#   st_transform(crs = 32614)|>                                                         #Convertir a WGS 84 UTM zona 14N
#   mutate(airbnb = 1)|>
#   st_join(ageb_2020, join = st_within, left = TRUE)|>                                 #Unir la base de airbnb con la de las AGEBS
#   st_drop_geometry()|>                                                                #Eliminar la geometría
#   group_by(CVEGEO)|>                                                                  #Agrupar por CVEGEO
#   summarise(unidades_airbnb = sum(airbnb))                                            #Contar el número de airbnbs por CVEGEO

room_type,alojamientos
<chr>,<int>
Entire home/apt,17430


In [9]:
#Cálculo de la población total por AGEBS
total_airbn<-cuenta_airbnb$alojamientos  
total_airbn                                     
total_vivparhab<-sum(RESAGEBURB_09XLSX20$TVIVPARHAB, na.rm = TRUE)                   #Total de viviendas particulares habitadas         
total_vivtot<-sum(RESAGEBURB_09XLSX20$VIVTOT, na.rm = TRUE)                                    
pobl_tot <-sum(RESAGEBURB_09XLSX20$POBTOT, na.rm = TRUE)                             #Total de población total

[1] 17430

In [ ]:
airbnb_por_mil_viv = total_airbn/total_vivparhab*1000 
cat("ALOJAMIENTOS AIRBNB POR CADA MIL VIVIENDAS PARTICULARES HABITADAS EN LA CIUDAD DE MÉXICO:", airbnb_por_mil_viv, "\n")
airbnb_por_mil_hab<-total_airbn/pobl_tot*1000
cat("ALOJAMIENTOS AIRBNB POR CADA MIL HABITANTES EN LA CIUDAD DE MÉXICO:", airbnb_por_mil_hab, "\n")


ALOJAMIENTOS AIRBNB POR CADA MIL VIVIENDAS PARTICULARES HABITADAS EN LA CIUDAD DE MÉXICO: 6.362199 
ALOJAMIENTOS AIRBNB POR CADA MIL HABITANTES EN LA CIUDAD DE MÉXICO: 1.905828 
ALOJAMIENTOS AIRBNB POR CADA MIL VIVIENDAS PARTICULARES HABITADAS EN LA CIUDAD DE MÉXICO: 6.362199 
